In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
 
warnings.filterwarnings("ignore")
 
from google.colab import drive  
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/MyDrive/dataset_merged_sorted.zip

Archive:  gdrive/MyDrive/dataset_merged_sorted.zip
  inflating: dataset_merged_sorted.csv  


In [ ]:
# df=pd.read_csv('/content/gdrive/MyDrive/dataset_sample_sorted.csv')
df=pd.read_csv('dataset_merged_sorted.csv')
len(df)

11263001

In [ ]:
# new_sample=pd.read_csv('/content/gdrive/MyDrive/dataset_sample.csv')
df=df.drop(columns=['time',	'physical_payload',	
                                'crc_status',	'frequency',	'bandwidth',	'code_rate'	,	'snr',	'size'	,'mtype',	'rssi','fport','spreading_factor'],axis=1)
df.head()

,device_address,gateway,fcnt
0,-1,00000f0c210281c4,-1
1,-1,00000f0c210281c4,-1
2,-1,00000f0c210281c4,-1
3,-1,00000f0c22433141,-1
4,-1,00000f0c22433141,-1


In [ ]:
df=df[(df.values ==-1).sum(axis=1) == 0]
df=df.dropna()
len(df)

8209871

In [ ]:
df=df.reset_index(drop=True)
df.head()

,device_address,gateway,fcnt
0,00123d57,00000f0c210721f2,1037
1,00123d57,00000f0c210721f2,1037
2,00123d57,00000f0c210281c4,1037
3,00123d57,00000f0c210281c4,1037
4,00aaba54,00000f0c210281c4,1648


In [ ]:
gateway_id=df.gateway.unique()
add_id=df.device_address.unique()


In [ ]:
def packet_lost(df,g):
  lostpacket=0
  z=(df.groupby(by=['gateway']).get_group(g))
  add_id=z.device_address.unique()
  for d in add_id:
    w=((z.groupby(by=['device_address']).get_group(d)['fcnt']).sort_values()).reset_index(drop=True)
    for i in range(len(w)-1):
      if (w[i+1]-w[i]) == 2:
          lostpacket = lostpacket+1
    del w
  return lostpacket , len(z)


In [ ]:
gateway_id = ['00000f0c22433141','00800000a0001793']

In [ ]:
lost_1,len_1=packet_lost(df,gateway_id[0])
ratio1=lost_1/len_1
print( 'ratio of lost packets for gateway 00000f0c22433141 is: ' , str(ratio1 * 100) + '%' )


ratio of lost packets for gateway 00000f0c22433141 is:  7.355167619341882%


In [ ]:
lost_2,len_2=packet_lost(df,gateway_id[1])
ratio1=lost_2/len_2
print( 'ratio of lost packets for gateway 00800000a0001793 is: ' , str(ratio2 * 100) + '%' )

In [ ]:
c=0
for i in gateway_id:
   c=c+ packet_lost(df,i)
c

2175

In [ ]:
def packets_lost(df,g):
  c=0
  z=(df.groupby(by=['gateway']).get_group(g))
  add_id=z.device_address.unique()
  for d in add_id:
    lostpacket=0
    w=z.groupby(by=['device_address']).get_group(d).fcnt.sort_values().reset_index(drop=True)
    lostpacket=[lostpacket+1 for i in range(len(w)-1) if w[i+1]-w[i] == 2]
    c=c+sum(lostpacket)
  return(c)

In [ ]:
c=0
for i in gateway_id:
   c=c+ packets_lost(df,i)
c

2175

In [ ]:
#number of lostpacket
# lostpacket

In [ ]:
#ratio of lostpacket
 
ratio = c/len(df)
print( 'ratio of lost packets is: ' , str(ratio * 100) + '%' )

ratio of lost packets is:  10.945599114287152%
